## Preparing the data : Monolingual data (MLM) and/or Parallel data (TLM)

In [100]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME 

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [101]:
%env PARA=True          
%env MONO=True           
                   
%env PARA_PATH=data/enfrde/para      
%env MONO_PATH=data/enfrde/mono    
%env SAME_VOCAB=True    
%env nCodes=10
%env shuf_n_samples=1
%env threads_for_tokenizer=16
%env test_size=10       
%env val_size=10        

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=data/enfrde/processed 
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=en-fr:10,de-en:-1,de-fr:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=data/enfrde/para
env: MONO_PATH=data/enfrde/mono
env: SAME_VOCAB=True
env: nCodes=10
env: shuf_n_samples=1
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=data/enfrde/processed
env: n_samples=-1
env: sub_task=en-fr:10,de-en:-1,de-fr:-1


In [3]:
! ../build_meta_data.sh $sub_task $n_samples 

dir data/enfrde/processed/fine_tune already exists
params ok !
*** Cleaning and tokenizing en-fr data ... ***
file data/enfrde/para/en-fr.en.all already exists
file data/enfrde/para/en-fr.fr.all already exists
*** Cleaning and tokenizing de-en data ... ***
file data/enfrde/para/de-en.de.all already exists
file data/enfrde/para/de-en.en.all already exists
*** Cleaning and tokenizing de-fr data ... ***
file data/enfrde/para/de-fr.de.all already exists
file data/enfrde/para/de-fr.fr.all already exists
file data/enfrde/mono/en.all already exists
file data/enfrde/mono/fr.all already exists
file data/enfrde/mono/de.all already exists
file data/enfrde/mono/en.all already exists
file data/enfrde/mono/de.all already exists
file data/enfrde/mono/fr.all already exists


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write 

## Training

In [149]:
%env exp_id=maml

env: exp_id=maml


In [150]:
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


### Pretrain a language meta-model (MLM + TLM)

In [152]:
%env lgs=en-fr|de-en|de-fr
%env batch_size=2
%env max_epoch=1
%env epoch_size=100

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=80
%env valid_n_samples=10
%env test_n_samples=10
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: lgs=en-fr|de-en|de-fr
env: batch_size=2
env: max_epoch=1
env: epoch_size=100
env: train_n_samples=80
env: valid_n_samples=10
env: test_n_samples=10
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [144]:
# MLM
#%env mlm_steps=en,fr|en,de|de,fr
# TML
#%env mlm_steps=en-fr|en-de|de-fr
# MLM + TML
%env mlm_steps=en,fr,en-fr|en,de,en-de|de,fr,de-fr

! python train.py --exp_name mlm_enfrde --exp_id $exp_id --dump_path ./dumped/ --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=en,fr,en-fr|en,de,en-de|de,fr,de-fr
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/15/20 08:40:28 - 0:00:00 - ============ Initialized logger ============
INFO - 05/15/20 08:40:28 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: [[], [], []]
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 2
                                     beam_size: 1
                                     bptt: 256
                                     bt_src_langs: []


## Train a (unsupervised/supervised) MT from a pretrained meta-model

In [156]:
%env lgs=en-fr|de-en|de-fr
%env batch_size=1
%env max_epoch=1
%env epoch_size=100

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=80
%env valid_n_samples=10
%env test_n_samples=10
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: lgs=en-fr|de-en|de-fr
env: batch_size=1
env: max_epoch=1
env: epoch_size=100
env: train_n_samples=80
env: valid_n_samples=10
env: test_n_samples=10
env: eval_bleu=true


In [154]:
%env stopping_criterion=valid_mt_bleu,5
%env validation_metrics=valid_mt_bleu
%env reload_model=dumped/mlm_enfrde/maml/best-valid_mlm_ppl.pth,dumped/mlm_enfrde/maml/best-valid_mlm_ppl.pth
%env ae_steps=en,fr|de,en|de,fr
%env bt_steps=en-fr-en,fr-en-fr|de-en-de,en-de-en|de-fr-de,fr-de-fr

env: stopping_criterion=valid_mt_bleu,5
env: validation_metrics=valid_mt_bleu
env: reload_model=dumped/mlm_enfrde/maml/best-valid_mlm_ppl.pth,dumped/mlm_enfrde/maml/best-valid_mlm_ppl.pth
env: ae_steps=en,fr|de,en|de,fr
env: bt_steps=en-fr-en,fr-en-fr|de-en-de,en-de-en|de-fr-de,fr-de-fr


In [157]:
# unsupervised MT
#! python train.py --exp_name meta_MT_enfrde --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

# supervised MT
%env mt_steps=en-fr,fr-en|de-en,en-de|de-fr,fr-de           
! python train.py --exp_name meta_MT_enfrde --exp_id $exp_id  --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: mt_steps=en-fr,fr-en|de-en,en-de|de-fr,fr-de
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/15/20 08:50:43 - 0:00:00 - ============ Initialized logger ============
INFO - 05/15/20 08:50:43 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: [['en', 'fr'], ['de', 'en'], ['de', 'fr']]
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 1
                                     beam_size: 1
                                     bptt: 256
                         

## Fine-tune the meta-model on a specific (sub) nmt (meta) task : en-fr

In [134]:
#%env batch_size=...
#%env epoch_size=...

%env lgs=en-fr

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: lgs=en-fr
env: eval_bleu=true


In [135]:
%env stopping_criterion=valid_en-fr_mt_bleu,5
%env validation_metrics=valid_en-fr_mt_bleu
%env reload_model=dumped/meta_MT_enfrde/maml/best-valid_mt_bleu.pth,dumped/meta_MT_enfrde/maml/best-valid_mt_bleu.pth
%env ae_steps=en,fr
%env bt_steps=en-fr-en,fr-en-fr

env: stopping_criterion=valid_en-fr_mt_bleu,5
env: validation_metrics=valid_en-fr_mt_bleu
env: reload_model=dumped/meta_MT_enfrde/maml/best-valid_mt_bleu.pth,dumped/meta_MT_enfrde/maml/best-valid_mt_bleu.pth
env: ae_steps=en,fr
env: bt_steps=en-fr-en,fr-en-fr


In [136]:
%env train_n_samples=8
%env valid_n_samples=2
%env test_n_samples=2

%env max_epoch=1   
%env batch_size=1
%env epoch_size=20

# unsupervised MT
#! python train.py --exp_name meta_MT_enfr --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH/fine_tune --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples #--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test  

# supervised MT
%env mt_steps=en-fr,fr-en           
! python train.py --exp_name meta_MT_enfr --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH/fine_tune --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples #--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test      

env: train_n_samples=8
env: valid_n_samples=2
env: test_n_samples=2
env: max_epoch=1
env: batch_size=1
env: epoch_size=20
env: mt_steps=en-fr,fr-en
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/15/20 08:20:08 - 0:00:00 - ============ Initialized logger ============
INFO - 05/15/20 08:20:08 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['en', 'fr']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 1
                                     beam_size: 1
    